In [1]:
import sys
import os
os.chdir("..")
import pandas as pd

In [2]:
train = pd.read_csv('data/feature/X_train.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
missing_values = train.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [5]:
train.shape

(18429, 82)

In [6]:
val = pd.read_csv('data/feature/X_val.csv')
test = pd.read_csv('data/feature/X_test.csv')

In [7]:
val.shape

(3270, 82)

In [8]:
test.shape

(3262, 82)

In [9]:
train.columns

Index(['league_id', 'stage', 'home_team_api_id', 'away_team_api_id', 'B365H',
       'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH',
       'LBD', 'LBA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'match_year',
       'match_month', 'home_wins_last5', 'home_avg_gs_last5',
       'home_avg_gc_last5', 'away_wins_last5', 'away_avg_gs_last5',
       'away_avg_gc_last5', 'h2h_wins_last3', 'h2h_nobs', 'h2h_win_rate_last3',
       'home_buildUpPlaySpeed', 'home_buildUpPlayPassing',
       'home_chanceCreationPassing', 'home_chanceCreationShooting',
       'home_defencePressure', 'home_defenceAggression',
       'home_defenceTeamWidth', 'away_buildUpPlaySpeed',
       'away_buildUpPlayPassing', 'away_chanceCreationPassing',
       'away_chanceCreationShooting', 'away_defencePressure',
       'away_defenceAggression', 'away_defenceTeamWidth', 'diff_speed',
       'diff_shooting', 'diff_pressure', 'away_avg_height', 'home_avg_height',
       'away_avg_weight', 'home_avg_weight'

In [10]:
df_2 = pd.read_csv('data/processed/df_2.csv')

In [11]:
df_2.shape

(24961, 85)